Prompt Chaining

In [ ]:
# ✅ Imports
from langgraph.graph import StateGraph, START, END   # StateGraph = workflow builder
from langchain_huggingface import HuggingFacePipeline  # wrapper to use HF models in LangChain
from typing import TypedDict   # define workflow state schema
from dotenv import load_dotenv # load env variables (like Hugging Face token)

load_dotenv()

gemma_model = "google/gemma-2-2b-it" 

model = HuggingFacePipeline.from_model_id(
    model_id=gemma_model,
    task="text-generation",   # type of pipeline we want
    pipeline_kwargs=dict(
        temperature=0.5,      # controls randomness (lower = deterministic)
        max_new_tokens=100,   # maximum tokens to generate per response
        return_full_text=False # return only the answer (not the prompt+answer)
    ),
)

class BlogState(TypedDict):
    
    title: str
    outline: str
    content: str
    
    
graph = StateGraph(BlogState)


def create_outline(state: BlogState) -> BlogState:

    title = state['title']
    
    prompt = f"Create a detailed outline for a blog post titled: {title}"
    
    outline = model.invoke(prompt)
    
    state['outline'] = outline

    return state





def create_blog(state: BlogState) -> BlogState:
    
    title = state['title']
    outline = state['outline']
    
    prompt = f"Write a detailed blog post based on the title: {title} and the outline: {outline}"
    
    content = model.invoke(prompt)
    
    state['content'] = content
    
    return state


graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)


graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)  

workflow = graph.compile()


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.73it/s]
Device set to use cuda:0


NameError: name 'StateFraph' is not defined

In [5]:
import torch, gc

# Delete any model or large tensors you no longer need
del model  # or any other big object
gc.collect()  # Python garbage collection
torch.cuda.empty_cache()  # clears cached memory
torch.cuda.ipc_collect()  # clears interprocess memory handles